In [1]:
import sys
!{sys.executable} -m pip install nltk

In [2]:
!{sys.executable} -m pip install textblob

In [12]:
import sys
!{sys.executable} -m pip install autocorrect

     |████████████████████████████████| 1.8MB 6.3MB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-1.1.0-cp37-none-any.whl size=1810770 sha256=5984c986a62fb7264d4c2b660ec4899ef73c3a7536cf1cf6329e190253a290dc
  Stored in directory: /home/tientn6/.cache/pip/wheels/78/7f/b1/527522820ae623df6a2dbe14f778d23adaea4bebe43f7ebcfe
Successfully built autocorrect


In [1]:
#Prevent Large File crashing on Jupyter Notebook
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [16]:
import pandas as pd
import numpy as np
from textblob import TextBlob, Word
from matplotlib.font_manager import FontProperties
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import datetime
import warnings 
#sns.set_style("darkgrid",{"axes.axisbelow" : False })
warnings.simplefilter('ignore')
import string
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer
import string   
import re
from nltk.stem import WordNetLemmatizer

ModuleNotFoundError: No module named 'textblob'

In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
STOPWORDS = stopwords.words("english") #stopwords are the most common unnecessary words. eg is, he, that, etc.

[nltk_data] Downloading package stopwords to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package punkt to C:\Users\Tien
[nltk_data]     Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Tien Nguyen\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [5]:
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii') # A function to remove emojis from the reviews

In [6]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [7]:
def clean_text(text):
    #ps=PorterStemmer()
    #wordnet_lemmatizer = WordNetLemmatizer()
    if not isinstance(text, str):
        return ''
    text=deEmojify(text) # remove emojis
    text_cleaned="".join([x for x in text if x not in string.punctuation]) # remove punctuation
    
    text_cleaned=re.sub(' +', ' ', text_cleaned) # remove extra white spaces
    text_cleaned=text_cleaned.lower() # converting to lowercase
    tokens=text_cleaned.split(" ")
    tokens=[token for token in tokens if token not in STOPWORDS] # Taking only those words which are not stopwords
    
    #Converting to lemma
    #text_cleaned=" ".join([wordnet_lemmatizer.lemmatize(token) for token in tokens])
    #text_cleaned=" ".join([ps.stem(token) for token in tokens])
    text_cleaned = lemmatize_with_postag(str(tokens))
    for r in (("\' ", ''), ('\'', ''), ('[',''),  (']','')):
        text_cleaned = text_cleaned.replace(*r)
    #print(text_cleaned)                            
    return text_cleaned

# ---------------------------------------------------
# Cleaning Amazon text reviews
 - remove emojis
 - remove punctuation
 - remove remove extra white spaces
 - converting to lowercase 
 - remove stopwords
 - using lemma instead of stem   
  _lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors._

In [8]:
### l = []
with open('amazon_official.tsv', 'r') as f:
    next(f)
    with open('amazon_cleaned_Official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t'+'reviewText' + '\t' + 'cleaned_text' + '\n')
        for i, line in enumerate(f):
            #if i == 3:
                #break
            line = line.strip().split('\t')
            overall = line[0]
            reviewTime = line[1]
            asin = line[2]
            if len(line) > 3:
                reviewText = line[3] 
                cleaned_text = clean_text(reviewText)
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + reviewText + '\t' + cleaned_text + '\n')
            else:
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + '' + '\t' + '' + '\n')

In [11]:
raw_data = pd.read_csv('amazon_cleaned_Official.tsv', sep = '\t', nrows=3)

In [14]:
pd.options.display.max_colwidth = 200
raw_data

,overall,reviewTime,asin,reviewText,cleaned_text
0,5.0,08 12 2005,1713353,This book is a winner with both of my boys. They really enjoy the pictures and the story. It's a classic.,book winner boy really enjoy picture story classic
1,5.0,03 30 2005,1713353,"The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book. It is one that I well remember from my own childhood and purchased for my daughter who loves it. It is about a ...",king mouse cheese nancy gurney excellent childrens book one well remember childhood purchased daughter love king trouble rude mouse eating cheese consults wise men suggest cat chase away mouse cat...
2,5.0,04 4 2004,1713353,"My daughter got her first copy from her great-grandmother (it had been my father's when he was a child). She loves it so much that she has worn out two copies now. Colorful pictures, easy to follo...",daughter got first copy greatgrandmother father child love much worn two copy colorful picture easy follow word wonderful moral sharing thinking problem easy way isnt always best way


# ---------------------------------------------------------------------
# More cleaning text
 - Contraction convert
 - Spelling check
 - Remove number

In [61]:
from autocorrect import Speller
import re
import nltk
spell = Speller(lang='en')
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}
def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def main():
    text = """I ain't going there. You'll have to go alone."""
    
    text=expand_contractions(text,contractions_dict)
    sentences = nltk.sent_tokenize(text)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    
    print (tokenized_sentences)

def more_clean_text(text):
    if not isinstance(text, str):
        return ''
    output = ''.join(c for c in text if not c.isdigit())
    text=expand_contractions(output,contractions_dict)
    word_tokens = nltk.word_tokenize(text)
    spells = ' '.join(spell(w) for w in (word_tokens))
    return spells

In [ ]:
### l = []
with open('amazon_cleaned_Official.tsv', 'r') as f:
    next(f)
    with open('amazon_cleaned_official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t'+'reviewText' + '\t' + 'cleaned_text' + '\n')
        for i, line in enumerate(f):
            #if i == 3:
                #break
            line = line.strip().split('\t')
            overall = line[0]
            reviewTime = line[1]
            asin = line[2]
            if len(line) > 4:
                reviewText = line[4] 
                cleaned_text = more_clean_text(reviewText)
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + reviewText + '\t' + cleaned_text + '\n')
            else:
                g.write(overall + '\t' + reviewTime + '\t' + asin +'\t' + '' + '\t' + '' + '\n')